In [38]:
#importing numpy, pandas, sklearn, pyplot, sqlalchemy 
import pandas as pd #for dataframe
import numpy as np #linear algebra libraries
import matplotlib.pyplot as plt #visualization
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import statsmodels.api as sm
from sqlalchemy import create_engine # for open connection of SQL and python
import warnings

warnings.filterwarnings('ignore')
#openning postgre for houseprices table
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [39]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houses = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()




In [40]:
#lets copy the file first
houses_c = houses.copy()

In [41]:
houses_c["is_Pave"] = pd.get_dummies(houses.street, drop_first=True)
houses_c["is_AllPub"] = pd.get_dummies(houses.utilities, drop_first=True)
houses_c["Y"] = pd.get_dummies(houses.centralair, drop_first=True)


In [42]:
dummy = pd.get_dummies(houses_c[['mszoning', 'lotshape', 'landcontour', 'lotconfig', 'landslope',
              'neighborhood', 'condition1', 'condition2', 'bldgtype', 'housestyle', 'roofstyle', 'roofmatl', 
              'exterior1st', 'exterior2nd', 'masvnrtype', 'exterqual', 'extercond', 'foundation', 'bsmtqual', 
              'bsmtexposure', 'bsmtfintype1', 'bsmtfintype2', 'heating', 'heatingqc', 'electrical',
              'kitchenqual', 'functional', 'garagetype', 'garagefinish', 'garagequal', 'garagecond',
              'paveddrive', 'saletype', 'salecondition']])

In [43]:
houses_c = pd.concat([houses_c, dummy], axis=1)

In [44]:
houses_c.columns

Index(['id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'street',
       'alley', 'lotshape', 'landcontour', 'utilities',
       ...
       'saletype_ConLw', 'saletype_New', 'saletype_Oth', 'saletype_WD',
       'salecondition_Abnorml', 'salecondition_AdjLand',
       'salecondition_Alloca', 'salecondition_Family', 'salecondition_Normal',
       'salecondition_Partial'],
      dtype='object', length=308)

In [45]:
houses_c['totrmsabvgrd_overallqual'] = houses_c['totrmsabvgrd'] * houses_c['overallqual']

In [46]:
houses_c = houses_c[["saleprice", "overallqual",  "totrmsabvgrd", "bsmtqual_Ex", "totrmsabvgrd_overallqual"]]

### OLS:

In [47]:


Y = houses_c['saleprice']


X = houses_c[["overallqual",  "totrmsabvgrd", "bsmtqual_Ex", "totrmsabvgrd_overallqual"]]

lrm = linear_model.LinearRegression()
# fit method estimates the coefficients using OLS
lrm.fit(X, Y)

# We add a constant to the model as it's a best practice
# to do so every time!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(Y, X).fit()

# We print the summary results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.731
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     991.0
Date:                Tue, 28 Jan 2020   Prob (F-statistic):               0.00
Time:                        13:10:12   Log-Likelihood:                -17584.
No. Observations:                1460   AIC:                         3.518e+04
Df Residuals:                    1455   BIC:                         3.520e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     7.598e+04   1.78e+04      4.273      0.000    4.11e+04    1.11e+05
overallqual               4335.2370   2911.903      1.489      0.137   -1376.739       1e+04
totrmsabvgrd             -1.692e+04   2721.020     -6.217      0.000   -2.23e+04   -1.16e+04
bsmtqual_Ex               5.058e+04   4602.414     10.991      0.000    4.16e+04    5.96e+04
totrmsabvgrd_overallqual  4533.2312    422.684     10.725      0.000    3704.096    5362.367
==============================================================================
Omnibus:                      288.026   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6310.601
Skew:                           0.287   Prob(JB):                         0.00
Kurtosis:                      13.169   Cond. No.                         757.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
#Accuracy:
from sklearn.model_selection import cross_val_score
score = cross_val_score(lrm, X, Y, cv=5)
print("OLS Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

OLS Accuracy: 0.73 (+/- 0.09)


### KNN:

#### Finding best hyperparameter:

In [50]:
from sklearn import neighbors
#Validation:
#importing modules
from sklearn.model_selection import GridSearchCV
#making the instance
knn = neighbors.KNeighborsRegressor(n_jobs=-1)
model = knn
#Hyper Parameters Set
params = {'n_neighbors':[5,6,7,8,9,10,11, 15, 18, 20],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=1)
#Learning
model1.fit(X,Y)
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)

Best Hyper Parameters:
 {'algorithm': 'brute', 'leaf_size': 1, 'n_jobs': -1, 'n_neighbors': 8, 'weights': 'uniform'}


In [52]:
from sklearn import neighbors
knn = neighbors.KNeighborsRegressor(n_neighbors=8, leaf_size = 1, algorithm = 'brute')
knn.fit(X, Y)

KNeighborsRegressor(algorithm='brute', leaf_size=1, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                    weights='uniform')

In [53]:
score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

Unweighted Accuracy: 0.71 (+/- 0.09)


In [61]:
print("KNN R2 score: %0.2f " % knn.score(X,Y))

KNN R2 score: 0.73 


OBS:
- Mean accuracy is higher for OLS than KNN, variance looks same for both, and R2 for both is same.
- Only looking at mean accuracy, I will choose OLS over KNN, also because OLS is great for bigger data than KNN.